In [ ]:
# imports
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch
import torchvision
import torchmetrics
import time

In [ ]:
# Utils

# Creating directory
def create_dir(addr):
    if not os.path.exists(addr):
        os.mkdir(addr)

# Delete folder and its content
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

# Addresses

# Data Address
data_address = "data"

# Raw data
raw_address = "../input/indian-birds/Birds_25"
raw_train = "../input/indian-birds/Birds_25/train"
raw_test = "../input/indian-birds/Birds_25/test"
raw_val = "../input/indian-birds/Birds_25/val"

# Processed data
processed_address = "data/processed"
cat_address = "data/processed/cat.pkl"
stat_address = "data/processed/stats.npz"

# Temp Address
temp_address = "temp"

# Results
result_address = "results/"
base_model_result = "results/base_model"

# Init Structure
create_dir(temp_address)
create_dir(data_address)
create_dir(processed_address)
create_dir(raw_address)
create_dir(raw_train)
create_dir(raw_test)
create_dir(raw_val)
create_dir(result_address)
# remove_folder_contents(result_address)

# Constants
random_seed = 68
n = 2
r = 25
batch_size = 32

# Setting Random Seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [ ]:
# Statistical Calculation

# Loading statistical data
def load_stat(stat_addr):
    x_stats = np.load(stat_addr)
    x_mean = x_stats['mean']
    x_std = x_stats['std']
    return x_mean, x_std

# Saving statistical data
def save_stat(lin_sum, quad_sum, stat_addr, total_ct):
    x_mean = (lin_sum/total_ct).astype(np.float64)
    x_std = np.sqrt(quad_sum/total_ct - x_mean**2).astype(np.float64)
    np.savez_compressed(stat_addr, mean = x_mean, std = x_std)
    print("Saved statistical processed data")

# Partition data into smaller fragments and returns sum and quad sum
def moment_data(category, shape, address, stat_addr, mode):
    # Reading Data and Preprocessing it
    lin_sum, quad_sum = np.zeros(shape, dtype=np.float64), np.zeros(shape, dtype=np.float64)
    ct_list = []
    for cat in category:
        cat_path = os.path.join(address, cat)
        ct_list.append(len(os.listdir(cat_path)))
        
        # Checking if already present
        if os.path.exists(stat_addr) or mode==1:
            continue
        
        # Reading each image of particular category
        count = 0
        for img_name in sorted(os.listdir(cat_path)):
            img_path = os.path.join(cat_path, img_name)
            img = np.array(cv2.imread(img_path))
            lin_sum += img.astype(np.float64)
            quad_sum += np.square(img.astype(np.float64))
            count += 1
        
        # log
        print(f"Read {cat}\tindex: {category.index(cat)}\tnum: {count}")

    if mode == 0:
        print("Read Train Data")
    else:
        print("Read Test Data")

    return lin_sum, quad_sum, ct_list

# Reading data fragments and normalizing them
def normalize(addr, category, processed_x_addr, processed_y_addr, norm, overwrite):
    ct = 0
    for cat in category:
        cat_addr = os.path.join(addr, cat)
        for img_name in sorted(os.listdir(cat_addr)):
            # Save Address
            x_addr = os.path.join(processed_x_addr, f'{ct}.pt')
            y_addr = os.path.join(processed_y_addr, f'{ct}.pt')
            if not overwrite and os.path.exists(x_addr) and os.path.exists(y_addr):
                ct += 1
                continue
            
            # Reading image
            img_path = os.path.join(cat_addr, img_name)
            img = np.array(cv2.imread(img_path), dtype=np.float64)
            
            # Normalizing image
            norm_img = norm(img)
            x = torch.tensor(norm_img)
            
            # Getting y
            y = np.zeros((len(category),), dtype=np.float64)
            y[category.index(cat)] = 1
            y = torch.Tensor(y)
            
            # Saving image
            torch.save(x, x_addr)
            torch.save(y, y_addr)
            
            # Incrementing counter
            ct += 1
        
        print(f"Preprocessed\tindex: {category.index(cat)}\t{cat}")

# Preprocessed data
def preprocess(address, mode, overwrite = False, cat_addr = None, stat_addr = None, shape = (256, 256, 3)):
    '''
    mode = 0: Training Mode
    mode = 1: Testing Mode
    '''

    if mode not in [0, 1]:
        raise Exception("Not a Valid Mode")

    # Identifying Categories
    if mode == 0:
        category = sorted(os.listdir(address))
        
        # Saving categories
        if overwrite or not os.path.exists(cat_addr):
            with open(cat_addr, 'wb') as cat_file:
                pickle.dump(category, cat_file)
        print("Assigned Categories")
        
    elif mode == 1:
        # Reading Category array
        with open(cat_addr, 'rb') as cat_file:
            category = pickle.load(cat_file)
        print("Read Categories")
    
    # count of images and their moments
    lin_sum, quad_sum, ct_list = moment_data(category=category, shape=shape, address=address, stat_addr=stat_addr, mode=mode)
        
    # Normalizing Data
    if mode == 0 and (overwrite or not os.path.exists(stat_addr)):
        save_stat(lin_sum=lin_sum, quad_sum=quad_sum, stat_addr=stat_addr, total_ct=sum(ct_list))
    x_mean, x_std = load_stat(stat_addr)
        
    # # Function for normalization
    # norm = lambda img: np.divide((img - x_mean), x_std, out = np.zeros_like(x_mean), where = x_std!=0)

    # # Creating train directory
    # create_dir(processed_x_addr)
    # create_dir(processed_y_addr)

    # # Reading Saved Files and Normalizing them
    # normalize(addr=address, category=category, processed_x_addr=processed_x_addr, processed_y_addr=processed_y_addr, norm=norm, overwrite=overwrite)

    return category

# Function Call
category = preprocess(address=raw_train, mode=0, overwrite=False, cat_addr=cat_address, stat_addr=stat_address)

In [ ]:
# cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working with {device}")

In [ ]:
# Data Loader

# Loading statistical data
x_mean, x_std = load_stat(stat_address)
x_mean = np.mean(np.mean(x_mean, axis=0), axis=0)
x_std = np.mean(np.mean(x_std, axis=0), axis=0)

# Transformation for preprocessing
transform_augment = torchvision.transforms.Compose([
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.RandomVerticalFlip(),
                    torchvision.transforms.RandomAutocontrast(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean = x_mean, std = x_std)
                    ])

transform_normal = torchvision.transforms.Compose([
                   torchvision.transforms.ToTensor(),
                   torchvision.transforms.Normalize(mean = x_mean, std = x_std)
                   ])

class DataSet(torch.utils.data.Dataset):
    def __init__(self, address, augment = True):
        self.address = address
        self.cat_list = []
        for cat in category:
            cat_path = os.path.join(self.address, cat)
            self.cat_list.append(sorted(os.listdir(cat_path)))
        self.transform = transform_augment if augment else transform_normal

    def __len__(self):
        return sum([len(elem) for elem in self.cat_list])
    
    def __getitem__(self, idx):
        ind = idx
        for cat_ind in range(len(self.cat_list)):
            if ind < len(self.cat_list[cat_ind]):
                cat_path = os.path.join(self.address, category[cat_ind])
                img_path = os.path.join(cat_path, self.cat_list[cat_ind][ind])
                y_vec = np.zeros((len(category),))
                y_vec[cat_ind] = 1
                y = torch.tensor(y_vec, dtype=torch.float64)
                break
            else:
                ind -= len(self.cat_list[cat_ind])
        img = torchvision.datasets.folder.default_loader(img_path)
        x = self.transform(img).to(torch.float64)
        return x, y

# Training Dataset and Data Loader
dataset_train = DataSet(raw_train)
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers = 4)

# Validation Dataset and Data Loader
dataset_val = DataSet(raw_val, augment=False)
data_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers = 4)

# Test Dataset and Data Loader
dataset_test = DataSet(raw_test, augment=False)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers = 4)

In [ ]:
# Model

class ResBlock(torch.nn.Module):
    def __init__(self, in_channel, out_channel, norm_func, kernel_size=3, stride=1):
        super(ResBlock, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(in_channel, in_channel, kernel_size, stride=1, padding=1, dtype=torch.float64)
        self.norm1 = norm_func(in_channel, dtype=torch.float64)
        self.activation1 = torch.nn.ReLU()
        
        self.conv2 = torch.nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding=1, dtype=torch.float64)
        self.norm2 = norm_func(out_channel, dtype=torch.float64)
        self.activation2 = torch.nn.ReLU()
        
        self.project = True if (in_channel != out_channel) or (stride != 1) else False
        if self.project:
            self.conv_project = torch.nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding=1, dtype=torch.float64)

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.activation1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x += self.conv_project(res) if self.project else res
        x = self.activation2(x)
        
        return x

class Resnet(torch.nn.Module):
    def __init__(self, n, r, norm_func = torch.nn.BatchNorm2d):
        super(Resnet, self).__init__()

        self.norm = norm_func
        
        #Input
        self.input_layer = []
        self.input_layer.append(torch.nn.Conv2d(3, 16, 3, 1, padding=1, dtype=torch.float64))
        self.input_layer.append(self.norm(16, dtype=torch.float64))
        self.input_layer.append(torch.nn.ReLU())
        self.input_layer = torch.nn.Sequential(*self.input_layer)
        
        # Layer1
        self.hidden_layer1 = []
        for i in range(n):
            self.hidden_layer1.append(ResBlock(16, 16, self.norm))
        self.hidden_layer1 = torch.nn.Sequential(*self.hidden_layer1)
        
        # Layer2
        self.hidden_layer2 = []
        self.hidden_layer2.append(ResBlock(16, 32, self.norm, stride = 2))
        for i in range(n-1):
            self.hidden_layer2.append(ResBlock(32, 32, self.norm))
        self.hidden_layer2 = torch.nn.Sequential(*self.hidden_layer2)
        
        # Layer3
        self.hidden_layer3 = []
        self.hidden_layer3.append(ResBlock(32, 64, self.norm, stride = 2))
        for i in range(n-1):
            self.hidden_layer3.append(ResBlock(64, 64, self.norm))
        self.hidden_layer3 = torch.nn.Sequential(*self.hidden_layer3)
            
        # Pool Layer
        self.pool = torch.nn.AdaptiveAvgPool2d(1)
        self.flatten = torch.nn.Flatten()
        
        # Output Layer
        self.output_layer = []
        self.output_layer.append(torch.nn.Linear(64, r, dtype=torch.float64))
        self.output_layer.append(torch.nn.Softmax(1))
        self.output_layer = torch.nn.Sequential(*self.output_layer)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer1(x)
        x = self.hidden_layer2(x)
        x = self.hidden_layer3(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.output_layer(x)
        
        return x


In [ ]:
# Training Model

def train(model, data_loader, save_addr, num_epoch = 50, learning_rate = 1e-2, overwrite = False):
    # Creating save folder
    create_dir(save_addr)
    model_addr = os.path.join(save_addr, 'model')
    create_dir(model_addr)
    loss_addr = os.path.join(save_addr, 'loss')
    create_dir(loss_addr)

    # Parameters for training
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    start_time = time.time()

    # Metrics
    metric_f1_micro = torchmetrics.classification.MulticlassF1Score(num_classes = r, average = 'micro').to(device)
    metric_f1_macro = torchmetrics.classification.MulticlassF1Score(num_classes = r, average = 'macro').to(device)
    metric_accuracy = torchmetrics.classification.Accuracy(task = 'multiclass', num_classes = r).to(device)
    
    for epoch in range(num_epoch):
        batch_ct = 0
        epoch_loss = 0
        label_arr = torch.tensor([], device=device)
        label_pred_arr = torch.tensor([], device=device)

        # Loading previous model
        epoch_addr = os.path.join(model_addr, f'{0 if epoch < 10 else ""}{epoch}.pt')
        epoch_loss_addr = os.path.join(loss_addr, f'{0 if epoch < 10 else ""}{epoch}.pt')

        if not overwrite and os.path.exists(epoch_addr) and os.path.exists(epoch_loss_addr):
            loss_arr = torch.load(epoch_loss_addr)
            epoch_loss = loss_arr[0].item()
            accuracy = loss_arr[1].item()
            f1_micro = loss_arr[2].item()
            f1_macro = loss_arr[3].item()

            print(f"Epoch: {epoch} Loaded\t\tLoss: {round(epoch_loss, 6)}\tAccuracy: {round(accuracy, 6)}\tf1_micro: {f1_micro}\tf1_macro: {f1_macro}")
        else:
            # Training next epoch
            for x, y in data_loader:
                # To Device
                x = x.to(device)
                y = y.to(device)

                # Predictions
                y_pred = model(x)

                # Maintaining label array
                label, label_pred = torch.argmax(y, dim=1), torch.argmax(y_pred, dim=1)
                label_arr = torch.cat((label_arr, label))
                label_pred_arr = torch.cat((label_pred_arr, label_pred))

                # Calculating Loss
                loss = loss_fn(y_pred, y)
                epoch_loss += loss.item()/label.numel()

                # Back Propogation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Log
                batch_ct += 1
                if batch_ct%100 == 0:
                    print(f"\tBatch: {batch_ct}\tTotal Loss: {round(epoch_loss/batch_ct, 6)}\tTime: {time.time()-start_time}")

            # Computing Metrics
            f1_micro = metric_f1_micro(label_pred_arr, label_arr).item()
            f1_macro = metric_f1_macro(label_pred_arr, label_arr).item()
            accuracy = metric_accuracy(label_pred_arr, label_arr).item()
            loss_arr = torch.tensor([epoch_loss/batch_ct, accuracy, f1_micro, f1_macro])

            # Saving model after each epoch
            torch.save(model.state_dict(), epoch_addr)
            torch.save(loss_arr, epoch_loss_addr)

            print(f"Epoch: {epoch}\tLoss: {round(epoch_loss/batch_ct, 6)}\tAccuracy: {round(accuracy, 6)}\tf1_micro: {f1_micro}\tf1_macro: {f1_macro}\tTime: {time.time() - start_time}")


In [ ]:
# Validate Model

def validate(model, data_loader, save_addr, load_addr, overwrite = False):
    # Creating save folder
    create_dir(save_addr)
    model_addr_load = os.path.join(load_addr, 'model')

    # Parameters for training
    loss_fn = torch.nn.CrossEntropyLoss()
    start_time = time.time()

    # Metrics
    metric_f1_micro = torchmetrics.classification.MulticlassF1Score(num_classes = r, average = 'micro').to(device)
    metric_f1_macro = torchmetrics.classification.MulticlassF1Score(num_classes = r, average = 'macro').to(device)
    metric_accuracy = torchmetrics.classification.Accuracy(task = 'multiclass', num_classes = r).to(device)

    epoch = 0
    for param_addr in sorted(os.listdir(model_addr_load)):
        # Save Address
        epoch_loss_addr = os.path.join(save_addr, param_addr)

        # Checking if already present
        if not overwrite and os.path.exists(epoch_loss_addr):
            loss_arr = torch.load(epoch_loss_addr)
            epoch_loss = loss_arr[0].item()
            accuracy = loss_arr[1].item()
            f1_micro = loss_arr[2].item()
            f1_macro = loss_arr[3].item()

            print(f"Epoch: {epoch} Loaded\t\tLoss: {round(epoch_loss, 6)}\tAccuracy: {round(accuracy, 6)}\tf1_micro: {f1_micro}\tf1_macro: {f1_macro}")
            epoch += 1
            continue
        
        # Initializing Variable
        batch_ct = 0
        epoch_loss = 0
        label_arr = torch.tensor([], device=device)
        label_pred_arr = torch.tensor([], device=device)

        # Loading Model
        model.load_state_dict(torch.load(os.path.join(model_addr_load, param_addr)))

        # Evaluate Model and freezing it
        model.eval()
        for param in model.parameters():
            param.requires_grad = False

        for x, y in data_loader:
            # To Device
            x = x.to(device)
            y = y.to(device)
            
            # Predictions
            y_pred = model(x)

            # Maintaining label array
            label, label_pred = torch.argmax(y, dim=1), torch.argmax(y_pred, dim=1)
            label_arr = torch.cat((label_arr, label))
            label_pred_arr = torch.cat((label_pred_arr, label_pred))

            # Calculating Loss
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()/label.numel()

            # Log
            batch_ct += 1
            if batch_ct%100 == 0:
                print(f"\tBatch: {batch_ct}\tTotal Loss: {round(epoch_loss/batch_ct, 6)}\tTime: {time.time()-start_time}")

        # Computing Metrics
        f1_micro = metric_f1_micro(label_pred_arr, label_arr).item()
        f1_macro = metric_f1_macro(label_pred_arr, label_arr).item()
        accuracy = metric_accuracy(label_pred_arr, label_arr).item()
        loss_arr = torch.tensor([epoch_loss/batch_ct, accuracy, f1_micro, f1_macro])

        # Saving model after each epoch
        torch.save(loss_arr, epoch_loss_addr)

        # Log
        print(f"Epoch: {epoch}\tLoss: {round(epoch_loss/batch_ct, 6)}\tAccuracy: {round(accuracy, 6)}\tf1_micro: {f1_micro}\tf1_macro: {f1_macro}\tTime: {time.time() - start_time}")
        epoch += 1


In [ ]:
# Plot
def save(arr_x, arr_train, arr_val, address, title, y_label):
    fig, ax = plt.subplots()
    ax.plot(arr_x, arr_train, label = 'Train')
    ax.plot(arr_x, arr_val, label = 'Validation')
    ax.set_xlabel("num_epochs")
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()
    plt.savefig(address)

def plot(train_address, val_address, save_address):
    # Loading metrics
    train_metrics = []
    val_metrics = []
    for epoch_name in sorted(os.listdir(train_address)):
        train_metrics.append(np.array(torch.load(os.path.join(train_address, epoch_name))))
        val_metrics.append(np.array(torch.load(os.path.join(val_address, epoch_name))))
    train_metrics = np.stack(train_metrics)
    val_metrics = np.stack(val_metrics)

    arr_x = np.arange(1, train_metrics.shape[0]+1)
    save(arr_x, train_metrics[:, 0], val_metrics[:, 0], os.path.join(save_address, 'loss'), "Cross Entropy Loss", "Loss")
    save(arr_x, 100*train_metrics[:, 1], 100*val_metrics[:, 1], os.path.join(save_address, 'acc'), "Accuracy", "Accuracy")
    save(arr_x, train_metrics[:, 2], val_metrics[:, 2], os.path.join(save_address, 'micro'), "F1 Micro Score", "f1_micro")
    save(arr_x, train_metrics[:, 3], val_metrics[:, 3], os.path.join(save_address, 'macro'), "F1 Macro Score", "f1_macro")

# Report
def report(address):
    metric_arr = []
    for folder_name in ['loss', 'val', 'test']:
        folder_address = os.path.join(address, folder_name)
        metric_addr = os.path.join(folder_address, sorted(os.listdir(folder_address))[-1])
        metric_arr.append(np.array(torch.load(metric_addr)))

    with open(os.path.join(address, 'result.txt'), 'w') as file:
        file.write("Training:\n")
        file.write(f"\tLoss:\t\t{metric_arr[0][0]}\n")
        file.write(f"\tAccuracy:\t{metric_arr[0][1]}\n")
        file.write(f"\tF1_Micro:\t{metric_arr[0][2]}\n")
        file.write(f"\tF1_Macro:\t{metric_arr[0][3]}\n")
        file.write("\nValidation:\n")
        file.write(f"\tLoss:\t\t{metric_arr[1][0]}\n")
        file.write(f"\tAccuracy:\t{metric_arr[1][1]}\n")
        file.write(f"\tF1_Micro:\t{metric_arr[1][2]}\n")
        file.write(f"\tF1_Macro:\t{metric_arr[1][3]}\n")
        file.write("\nTesting:\n")
        file.write(f"\tLoss:\t\t{metric_arr[2][0]}\n")
        file.write(f"\tAccuracy:\t{metric_arr[2][1]}\n")
        file.write(f"\tF1_Micro:\t{metric_arr[2][2]}\n")
        file.write(f"\tF1_Macro:\t{metric_arr[2][3]}\n")


In [ ]:
# Base Model

resnet_base_model = Resnet(n, r).to(device)
train(resnet_base_model, data_loader_train, base_model_result)
validate(resnet_base_model, data_loader_val, os.path.join(base_model_result, 'val'), base_model_result)
validate(resnet_base_model, data_loader_test, os.path.join(base_model_result, 'test'), base_model_result)
plot (os.path.join(base_model_result, 'loss'), os.path.join(base_model_result, 'val'), base_model_result)
report(base_model_result)
